### Tutorial 2: Adding a new monatomic species entry to your local data store, the automatic way

If you would like to do calculations using a plasma species that you haven't used before, you will need to generate a minplascalc data entry for it first. You only need to do this <i>once</i> - minplascalc will store the species data in a plain-text file formatted using JSON syntax, and it will then be available for use in any of your future calculations.

For monatomic species minplascalc has built-in readers for handling data obtained from the energy levels section of NIST Atomic Spectra Database, which can be found at http://physics.nist.gov/PhysRefData/ASD/levels_form.html. Since the format in which the ASD reports energy levels is fairly consistent, the data can be retrieved from the site using automated tools - special thanks to Carl Sandrock for contributing the scraping methods in the following code snippets.

Start by importing and initialising the required modules: request for handling HTTP requests, HTMLparser for prettifying the returned data, io for passing it to minplascalc's constructors, and obviously minplascalc itself.

In [1]:
import requests
from html.parser import HTMLParser
import minplascalc as mpc

Generate a request for the singly-charged oxygen cation O<sup>+</sup>, or "O II" in spectrography speak.

In [2]:
compoundname = 'O II'
url = "https://physics.nist.gov/cgi-bin/ASD/energy1.pl"
props = {'encodedlist': 'XXT2',
         'spectrum': compoundname,  # This is the compound name
         'submit': 'Retrieve Data',
         'units': 0,
         'format': 1,  # 0 for HTML 1 for ASCII
         'output': 0,
         'page_size': 15,
         'multiplet_ordered': 0,
         'level_out': 'on',
         'j_out': 'on',
         'temp': None}

Pull the data from NIST.

In [3]:
r = requests.post(url, data=props)
ascii_response = r.text

The following code snippet is copied from the documentation on html.parser, and generates a parser object for filtering the NIST page's ASCII response.

In [4]:
class MyHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.str = ''
        self.armed = False

        
    def handle_starttag(self, tag, attrs):
        if tag.lower() == 'pre':
            self.armed = True
            
    def handle_endtag(self, tag):
        if tag.lower() == 'pre':
            self.armed = False
        
    def handle_data(self, data):
        if self.armed:
            self.str += data
            
parser = MyHTMLParser()

Use the parser object to prune the extraneous content out of the NIST response.

In [5]:
parser.feed(ascii_response)

Pass the species data into minplascalc and build a local JSON data entry for it.

In [6]:
with open("O+.nisttemp", "w") as f:
    f.write(parser.str)
    
mpc.buildMonatomicSpeciesJSON(
    name="O+",
    stoichiometry={"O": 1},
    molarMass=0.0159994,
    chargeNumber=1,
    ionisationEnergy=283270.9,
    nistDataFile="O+.nisttemp")

What's happening here? First we write the NIST scraped data out to a temporary file "O+.nisttemp", and then we call the <span style="font-family: 'Courier';">buildMonatomicSpeciesJSON</span> function to build and save the species data entry for us. 

The function takes as arguments the species name (which is also used as the data entry's associated file name), a small dictionary describing the elemental stoichiometry of the species (in this case a single oxygen atom), the molar mass in kg/mol, the charge on the species in units of the fundamental charge (in this case 1 because O<sup>+</sup> is singly charged), the ionisation energy of the species in cm<sup>-1</sup>, and the path to the energy level text file we just generated from the NIST site. Internally to minplascalc the function reads the energy levels from the text file, assembles the species information into a Python dictionary format, and writes it out to a JSON file called "O+.json" on the path you're running this notebook from.

After this process it will be possible to create an O<sup>+</sup> species object in any minplascalc calculation by importing it using either the explicit path to the JSON file, or (preferably) just the name of the species provided the JSON file is stored in any of the standard minplascalc data paths - see later tutorials for examples.